# گردش کار انسان در حلقه با چارچوب عامل مایکروسافت

## 🎯 اهداف یادگیری

در این دفترچه، یاد خواهید گرفت که چگونه گردش کارهای **انسان در حلقه** را با استفاده از `RequestInfoExecutor` در چارچوب عامل مایکروسافت پیاده‌سازی کنید. این الگوی قدرتمند به شما امکان می‌دهد تا گردش کارهای هوش مصنوعی را متوقف کنید تا ورودی انسانی جمع‌آوری شود، و این کار عوامل شما را تعاملی کرده و کنترل تصمیمات حیاتی را به انسان‌ها می‌سپارد.

## 🔄 انسان در حلقه چیست؟

**انسان در حلقه (HITL)** یک الگوی طراحی است که در آن عوامل هوش مصنوعی اجرای خود را متوقف می‌کنند تا قبل از ادامه، ورودی انسانی درخواست کنند. این امر برای موارد زیر ضروری است:

- ✅ **تصمیمات حیاتی** - دریافت تأیید انسانی قبل از انجام اقدامات مهم
- ✅ **وضعیت‌های مبهم** - اجازه دهید انسان‌ها زمانی که هوش مصنوعی مطمئن نیست، توضیح دهند
- ✅ **ترجیحات کاربر** - از کاربران بخواهید بین گزینه‌های مختلف انتخاب کنند
- ✅ **رعایت قوانین و ایمنی** - اطمینان از نظارت انسانی برای عملیات‌های تحت نظارت
- ✅ **تجربه‌های تعاملی** - ساخت عوامل مکالمه‌ای که به ورودی کاربر پاسخ می‌دهند

## 🏗️ نحوه کار در چارچوب عامل مایکروسافت

این چارچوب سه مؤلفه کلیدی برای HITL ارائه می‌دهد:

1. **`RequestInfoExecutor`** - یک اجراکننده ویژه که گردش کار را متوقف کرده و یک `RequestInfoEvent` منتشر می‌کند
2. **`RequestInfoMessage`** - کلاس پایه برای پیام‌های درخواست تایپ‌شده که به انسان‌ها ارسال می‌شود
3. **`RequestResponse`** - پاسخ‌های انسانی را با درخواست‌های اصلی با استفاده از `request_id` مرتبط می‌کند

**الگوی گردش کار:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 مثال ما: رزرو هتل با تأیید کاربر

ما بر اساس گردش کار شرطی، تأیید انسانی را **قبل از** پیشنهاد مقصدهای جایگزین اضافه خواهیم کرد:

1. کاربر یک مقصد درخواست می‌کند (مثلاً "پاریس")
2. `availability_agent` بررسی می‌کند که آیا اتاق‌ها موجود هستند
3. **اگر اتاقی موجود نباشد** → `confirmation_agent` می‌پرسد "آیا مایلید گزینه‌های جایگزین را ببینید؟"
4. گردش کار با استفاده از `RequestInfoExecutor` **متوقف می‌شود**
5. **انسان پاسخ می‌دهد** "بله" یا "خیر" از طریق ورودی کنسول
6. `decision_manager` بر اساس پاسخ مسیر را تعیین می‌کند:
   - **بله** → نمایش مقصدهای جایگزین
   - **خیر** → لغو درخواست رزرو
7. نمایش نتیجه نهایی

این نشان می‌دهد که چگونه می‌توان کنترل پیشنهادات عامل را به کاربران سپرد!

---

بیایید شروع کنیم! 🚀


## مرحله ۱: وارد کردن کتابخانه‌های مورد نیاز

ما اجزای استاندارد چارچوب Agent را به همراه **کلاس‌های خاص مرتبط با انسان در حلقه** وارد می‌کنیم:
- `RequestInfoExecutor` - اجرایی که جریان کار را برای دریافت ورودی انسانی متوقف می‌کند
- `RequestInfoEvent` - رویدادی که هنگام درخواست ورودی انسانی صادر می‌شود
- `RequestInfoMessage` - کلاس پایه برای بارهای درخواست تایپ‌شده
- `RequestResponse` - ارتباط‌دهنده پاسخ‌های انسانی با درخواست‌ها
- `WorkflowOutputEvent` - رویدادی برای شناسایی خروجی‌های جریان کار


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## مرحله ۲: تعریف مدل‌های Pydantic برای خروجی‌های ساختاریافته

این مدل‌ها **طرح**ی را تعریف می‌کنند که عوامل بازگشت خواهند داد. ما تمام مدل‌ها را از جریان کاری شرطی حفظ می‌کنیم و اضافه می‌کنیم:

**جدید برای انسان در حلقه:**
- `HumanFeedbackRequest` - زیرکلاسی از `RequestInfoMessage` که بار درخواست ارسال شده به انسان‌ها را تعریف می‌کند
  - شامل `prompt` (سؤالی که باید پرسیده شود) و `destination` (زمینه‌ای درباره شهر غیرقابل دسترس)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## مرحله ۳: ایجاد ابزار رزرو هتل

همان ابزار از جریان کاری شرطی - بررسی می‌کند که آیا اتاق‌ها در مقصد موجود هستند.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## مرحله ۴: تعریف توابع شرطی برای مسیریابی

ما به **چهار تابع شرطی** برای جریان کاری انسانی در حلقه نیاز داریم:

**از جریان کاری شرطی:**
1. `has_availability_condition` - مسیریابی زمانی که هتل‌ها موجود هستند
2. `no_availability_condition` - مسیریابی زمانی که هتل‌ها موجود نیستند

**جدید برای انسانی در حلقه:**
3. `user_wants_alternatives_condition` - مسیریابی زمانی که کاربر به گزینه‌های جایگزین "بله" می‌گوید
4. `user_declines_alternatives_condition` - مسیریابی زمانی که کاربر به گزینه‌های جایگزین "نه" می‌گوید


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## مرحله ۵: ایجاد اجراکننده مدیر تصمیم‌گیری

این **هسته الگوی انسان در حلقه** است! `DecisionManager` یک `Executor` سفارشی است که:

1. **بازخورد انسانی را دریافت می‌کند** از طریق اشیاء `RequestResponse`
2. **تصمیم کاربر را پردازش می‌کند** (بله/خیر)
3. **جریان کار را هدایت می‌کند** با ارسال پیام‌ها به عوامل مناسب

ویژگی‌های کلیدی:
- از تزئین‌کننده `@handler` استفاده می‌کند تا روش‌ها را به عنوان مراحل جریان کار معرفی کند
- `RequestResponse[HumanFeedbackRequest, str]` را دریافت می‌کند که شامل درخواست اصلی و پاسخ کاربر است
- پیام‌های ساده "بله" یا "خیر" تولید می‌کند که توابع شرطی ما را فعال می‌کنند


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## مرحله ۶: ایجاد اجراکننده نمایش سفارشی

همان اجراکننده نمایش از جریان کاری شرطی - نتایج نهایی را به عنوان خروجی جریان کاری ارائه می‌دهد.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## مرحله ۷: بارگذاری متغیرهای محیطی

پیکربندی کلاینت LLM (مدل‌های GitHub، Azure OpenAI یا OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## مرحله ۸: ایجاد عوامل هوش مصنوعی و اجراکننده‌ها

ما **شش جزء کاری** ایجاد می‌کنیم:

**عوامل (در AgentExecutor پیچیده شده‌اند):**
1. **availability_agent** - بررسی موجودی هتل با استفاده از ابزار
2. **confirmation_agent** - 🆕 آماده‌سازی درخواست تأیید از انسان
3. **alternative_agent** - پیشنهاد شهرهای جایگزین (وقتی کاربر می‌گوید بله)
4. **booking_agent** - تشویق به رزرو (وقتی اتاق‌ها موجود هستند)
5. **cancellation_agent** - 🆕 مدیریت پیام لغو (وقتی کاربر می‌گوید نه)

**اجراکننده‌های ویژه:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` که جریان کار را برای دریافت ورودی انسانی متوقف می‌کند
7. **decision_manager** - 🆕 اجراکننده سفارشی که بر اساس پاسخ انسانی مسیر را تعیین می‌کند (قبلاً در بالا تعریف شده است)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## مرحله ۹: ساخت جریان کاری با حضور انسان در حلقه

اکنون نمودار جریان کاری را با **مسیرهای شرطی** شامل مسیر حضور انسان در حلقه ایجاد می‌کنیم:

**ساختار جریان کاری:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**مسیرهای کلیدی:**
- `availability_agent → confirmation_agent` (وقتی اتاقی موجود نیست)
- `confirmation_agent → prepare_human_request` (تغییر نوع)
- `prepare_human_request → request_info_executor` (توقف برای انسان)
- `request_info_executor → decision_manager` (همیشه - ارائه RequestResponse)
- `decision_manager → alternative_agent` (وقتی کاربر می‌گوید "بله")
- `decision_manager → cancellation_agent` (وقتی کاربر می‌گوید "نه")
- `availability_agent → booking_agent` (وقتی اتاق‌ها موجود هستند)
- همه مسیرها در `display_result` پایان می‌یابند


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## مرحله ۱۰: اجرای تست مورد ۱ - شهر بدون موجودی (پاریس با تأیید انسانی)

این تست چرخه کامل **انسان در حلقه** را نشان می‌دهد:

1. درخواست هتل در پاریس
2. بررسی توسط availability_agent → بدون اتاق
3. confirmation_agent سوالی برای انسان ایجاد می‌کند
4. request_info_executor **جریان کار را متوقف می‌کند** و `RequestInfoEvent` را ارسال می‌کند
5. **برنامه رویداد را تشخیص داده و کاربر را در کنسول درخواست می‌کند**
6. کاربر "بله" یا "خیر" تایپ می‌کند
7. برنامه پاسخ را از طریق `send_responses_streaming()` ارسال می‌کند
8. decision_manager بر اساس پاسخ مسیر را تعیین می‌کند
9. نتیجه نهایی نمایش داده می‌شود

**الگوی کلیدی:**
- استفاده از `workflow.run_stream()` برای اولین تکرار
- استفاده از `workflow.send_responses_streaming(pending_responses)` برای تکرارهای بعدی
- گوش دادن به `RequestInfoEvent` برای تشخیص زمانی که ورودی انسانی لازم است
- گوش دادن به `WorkflowOutputEvent` برای دریافت نتایج نهایی


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## مرحله 11: اجرای مورد آزمایشی 2 - شهر با موجودی (استکهلم - بدون نیاز به ورودی انسانی)

این آزمایش مسیر **مستقیم** را زمانی که اتاق‌ها موجود هستند نشان می‌دهد:

1. درخواست هتل در استکهلم
2. بررسی توسط availability_agent → اتاق‌ها موجود هستند ✅
3. booking_agent پیشنهاد رزرو می‌دهد
4. display_result تأییدیه را نشان می‌دهد
5. **نیازی به ورودی انسانی نیست!**

این جریان کاملاً مسیر انسانی در حلقه را زمانی که اتاق‌ها موجود هستند دور می‌زند.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## نکات کلیدی و بهترین روش‌ها برای انسان در حلقه

### ✅ آنچه آموختید:

#### 1. **الگوی RequestInfoExecutor**
الگوی انسان در حلقه در Microsoft Agent Framework از سه جزء اصلی استفاده می‌کند:
- `RequestInfoExecutor` - جریان کار را متوقف کرده و رویدادها را منتشر می‌کند
- `RequestInfoMessage` - کلاس پایه برای بارهای درخواست تایپ شده (این را زیرکلاس کنید!)
- `RequestResponse` - پاسخ‌های انسانی را با درخواست‌های اصلی مرتبط می‌کند

**درک حیاتی:**
- `RequestInfoExecutor` خودش ورودی جمع‌آوری نمی‌کند - فقط جریان کار را متوقف می‌کند
- کد برنامه شما باید به `RequestInfoEvent` گوش دهد و ورودی را جمع‌آوری کند
- شما باید `send_responses_streaming()` را با یک دیکشنری که `request_id` را به پاسخ کاربر نگاشت می‌کند، فراخوانی کنید

#### 2. **الگوی اجرای جریان**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **معماری مبتنی بر رویداد**
برای کنترل جریان کار به رویدادهای خاص گوش دهید:
- `RequestInfoEvent` - نیاز به ورودی انسانی (جریان کار متوقف شده)
- `WorkflowOutputEvent` - نتیجه نهایی در دسترس است (جریان کار کامل شده)
- `WorkflowStatusEvent` - تغییرات وضعیت (IN_PROGRESS، IDLE_WITH_PENDING_REQUESTS و غیره)

#### 4. **اجراکننده‌های سفارشی با @handler**
`DecisionManager` نشان می‌دهد که چگونه اجراکننده‌هایی ایجاد کنید که:
- از دکوریتور `@handler` برای نمایش متدها به عنوان مراحل جریان کار استفاده می‌کنند
- پیام‌های تایپ شده دریافت می‌کنند (مانند `RequestResponse[HumanFeedbackRequest, str]`)
- جریان کار را با ارسال پیام‌ها به اجراکننده‌های دیگر هدایت می‌کنند
- از طریق `WorkflowContext` به زمینه دسترسی دارند

#### 5. **مسیر شرطی با تصمیمات انسانی**
می‌توانید توابع شرطی ایجاد کنید که پاسخ‌های انسانی را ارزیابی کنند:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 کاربردهای واقعی:

1. **جریان‌های کاری تأیید**
   - دریافت تأیید مدیر قبل از پردازش گزارش‌های هزینه
   - نیاز به بررسی انسانی قبل از ارسال ایمیل‌های خودکار
   - تأیید تراکنش‌های با ارزش بالا قبل از اجرا

2. **نظارت بر محتوا**
   - علامت‌گذاری محتوای مشکوک برای بررسی انسانی
   - درخواست از ناظران برای تصمیم‌گیری نهایی در موارد مرزی
   - ارجاع به انسان‌ها زمانی که اعتماد به نفس هوش مصنوعی پایین است

3. **خدمات مشتری**
   - اجازه دهید هوش مصنوعی به طور خودکار به سوالات معمولی پاسخ دهد
   - مسائل پیچیده را به نمایندگان انسانی ارجاع دهید
   - از مشتری بپرسید که آیا می‌خواهد با یک انسان صحبت کند

4. **پردازش داده‌ها**
   - درخواست از انسان‌ها برای حل ورودی‌های داده مبهم
   - تأیید تفسیرهای هوش مصنوعی از اسناد نامشخص
   - اجازه دهید کاربران بین تفسیرهای معتبر مختلف انتخاب کنند

5. **سیستم‌های حساس به ایمنی**
   - نیاز به تأیید انسانی قبل از اقدامات غیرقابل برگشت
   - دریافت تأیید قبل از دسترسی به داده‌های حساس
   - تأیید تصمیمات در صنایع تنظیم‌شده (بهداشت و درمان، مالی)

6. **عامل‌های تعاملی**
   - ساخت ربات‌های مکالمه‌ای که سوالات پیگیری می‌پرسند
   - ایجاد راهنماهایی که کاربران را در فرآیندهای پیچیده هدایت می‌کنند
   - طراحی عامل‌هایی که مرحله به مرحله با انسان‌ها همکاری می‌کنند

### 🔄 مقایسه: با و بدون انسان در حلقه

| ویژگی | جریان کار شرطی | جریان کار انسان در حلقه |
|-------|----------------|-------------------------|
| **اجرا** | یک `workflow.run()` | حلقه با `run_stream()` + `send_responses_streaming()` |
| **ورودی کاربر** | هیچ (کاملاً خودکار) | درخواست‌های تعاملی از طریق `input()` یا رابط کاربری |
| **اجزاء** | عامل‌ها + اجراکننده‌ها | + RequestInfoExecutor + DecisionManager |
| **رویدادها** | فقط AgentExecutorResponse | RequestInfoEvent، WorkflowOutputEvent و غیره |
| **توقف** | بدون توقف | جریان کار در RequestInfoExecutor متوقف می‌شود |
| **کنترل انسانی** | بدون کنترل انسانی | انسان‌ها تصمیمات کلیدی می‌گیرند |
| **مورد استفاده** | اتوماسیون | همکاری و نظارت |

### 🚀 الگوهای پیشرفته:

#### نقاط تصمیم‌گیری انسانی متعدد
می‌توانید چندین گره `RequestInfoExecutor` در یک جریان کار داشته باشید:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### مدیریت زمان‌بندی
زمان‌بندی برای پاسخ‌های انسانی را پیاده‌سازی کنید:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### ادغام رابط کاربری غنی
به جای `input()`، با رابط وب، Slack، Teams و غیره ادغام کنید:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### انسان در حلقه شرطی
فقط در شرایط خاص درخواست ورودی انسانی کنید:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ بهترین روش‌ها:

1. **همیشه RequestInfoMessage را زیرکلاس کنید**
   - ایمنی نوع و اعتبارسنجی را فراهم می‌کند
   - زمینه غنی برای رندر رابط کاربری را فعال می‌کند
   - هدف هر نوع درخواست را روشن می‌کند

2. **از درخواست‌های توصیفی استفاده کنید**
   - شامل زمینه‌ای درباره آنچه می‌پرسید باشید
   - پیامدهای هر انتخاب را توضیح دهید
   - سوالات را ساده و واضح نگه دارید

3. **مدیریت ورودی غیرمنتظره**
   - پاسخ‌های کاربر را اعتبارسنجی کنید
   - برای ورودی نامعتبر مقادیر پیش‌فرض ارائه دهید
   - پیام‌های خطای واضح بدهید

4. **ردیابی شناسه‌های درخواست**
   - از ارتباط بین request_id و پاسخ‌ها استفاده کنید
   - سعی نکنید وضعیت را به صورت دستی مدیریت کنید

5. **طراحی برای غیرمسدودکننده بودن**
   - نخ‌ها را منتظر ورودی نگه ندارید
   - از الگوهای غیرهمزمان در سراسر استفاده کنید
   - از نمونه‌های جریان کار همزمان پشتیبانی کنید

### 📚 مفاهیم مرتبط:

- **Agent Middleware** - رهگیری تماس‌های عامل (دفترچه قبلی)
- **مدیریت وضعیت جریان کار** - حفظ وضعیت جریان کار بین اجراها
- **همکاری چندعاملی** - ترکیب انسان در حلقه با تیم‌های عامل
- **معماری‌های مبتنی بر رویداد** - ساخت سیستم‌های واکنشی با رویدادها

---

### 🎓 تبریک!

شما جریان‌های کاری انسان در حلقه را با Microsoft Agent Framework به خوبی یاد گرفتید! اکنون می‌دانید که چگونه:
- ✅ جریان‌های کاری را برای جمع‌آوری ورودی انسانی متوقف کنید
- ✅ از RequestInfoExecutor و RequestInfoMessage استفاده کنید
- ✅ اجرای جریان را با رویدادها مدیریت کنید
- ✅ اجراکننده‌های سفارشی با @handler ایجاد کنید
- ✅ جریان‌های کاری را بر اساس تصمیمات انسانی هدایت کنید
- ✅ عامل‌های هوش مصنوعی تعاملی بسازید که با انسان‌ها همکاری کنند

**این یک الگوی حیاتی برای ساخت سیستم‌های هوش مصنوعی قابل اعتماد و قابل کنترل است!** 🚀



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
